In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical

text = "HELLO"
chars = sorted(set(text))
char_to_index = {c: i for i, c in enumerate(chars)}
index_to_char = {i: c for c, i in char_to_index.items()}

input_chars = text[:-1]
target_chars = text[1:]

input_indices = [char_to_index[c] for c in input_chars]
target_indices = [char_to_index[c] for c in target_chars]

X = to_categorical(input_indices, num_classes=len(chars))
X = np.reshape(X, (len(X), 1, len(chars)))
y = to_categorical(target_indices, num_classes=len(chars))
y = np.reshape(y, (len(y), 1, len(chars)))

input_layer = Input(shape=(1, len(chars)))
rnn_output, final_state = SimpleRNN(8, return_sequences=True, return_state=True)(input_layer)
output = Dense(len(chars), activation='softmax')(rnn_output)

model = Model(inputs=input_layer, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

model.fit(X, y, epochs=500, verbose=0)

extract_hidden_model = Model(inputs=input_layer, outputs=rnn_output)
hidden_states = extract_hidden_model.predict(X)

print("\nIntermediate Hidden States:")
for t in range(len(input_chars)):
    print(f"{input_chars[t]} → {np.round(hidden_states[t][0], 3)}")

print("\nPredicted Next Characters:")
predictions = model.predict(X)
for i, pred in enumerate(predictions):
    predicted_index = np.argmax(pred[0])
    print(f"{input_chars[i]} → {index_to_char[predicted_index]}")


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 4)]            0         
                                                                 
 simple_rnn (SimpleRNN)      [(None, 1, 8),            104       
                              (None, 8)]                         
                                                                 
 dense (Dense)               (None, 1, 4)              36        
                                                                 
Total params: 140
Trainable params: 140
Non-trainable params: 0
_________________________________________________________________
1/1 [==============================] - 0s 298ms/step

Intermediate Hidden States:
H → [-0.675  0.249 -0.673 -0.153 -0.708 -0.562  0.603  0.66 ]
E → [-0.52  -0.721  0.647 -0.585  0.539  0.782  0.732 -0.113]
L → [-0.469 -0.779  0.804 -0.176  0.042  0.614 -0.093 -

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical

# Prepare text and mappings
text = "HELLO"
chars = sorted(set(text))
char_to_index = {c: i for i, c in enumerate(chars)}
index_to_char = {i: c for c, i in char_to_index.items()}

# Prepare input and target sequences
input_chars = text[:-1]
target_chars = text[1:]
input_indices = [char_to_index[c] for c in input_chars]
target_indices = [char_to_index[c] for c in target_chars]

# One-hot encode inputs and reshape
X = to_categorical(input_indices, num_classes=len(chars))
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))
y = to_categorical(target_indices, num_classes=len(chars))

# Create model using functional API to get hidden states
input_layer = Input(shape=(1, len(chars)))
rnn_layer, state = SimpleRNN(32, return_state=True)(input_layer)
output_layer = Dense(len(chars), activation='softmax')(rnn_layer)

# Full model (for training)
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(X, y, epochs=500, verbose=0)

# Separate model to extract hidden states
state_model = Model(inputs=input_layer, outputs=[output_layer, state])

# Prediction function with hidden state output
def predict_next_letter_with_state(char):
    input_idx = char_to_index[char]
    input_onehot = to_categorical([input_idx], num_classes=len(chars))
    input_onehot = np.reshape(input_onehot, (1, 1, len(chars)))
    prediction, hidden_state = state_model.predict(input_onehot, verbose=0)
    predicted_index = np.argmax(prediction)
    predicted_char = index_to_char[predicted_index]
    return predicted_char, hidden_state

# Run predictions with hidden states
print("Predictions and Hidden States:")
for c in input_chars:
    pred, h = predict_next_letter_with_state(c)
    print(f"{c} → {pred}, Hidden State: {np.round(h, 4)}")
